10.01.23

# Select slices that experts will annotate

*IDEA*: select a total of 10 slices
- 3 where the model perform poorly
    - if it is obvious why -> discard
    - if it is interesting -> keep
- 7 selected "randomly" among the top k frames with the most events annotated

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
#import glob
import os
import imageio
import napari
import matplotlib.pyplot as plt
import math
import seaborn as sns
import pandas as pd

#from skimage.io import imsave

from in_out_tools import load_movies_ids, load_annotations_ids
from visualization_tools import get_discrete_cmap, get_labels_cmap, get_annotations_contour
from metrics_tools import compute_iou, empty_marginal_frames
from data_processing_tools import get_processed_result, class_to_nb

## Set global parameters

In [3]:
# configure Napari cmap
cmap = get_discrete_cmap(name='gray', lut=16)
labels_cmap = get_labels_cmap()

# frames ignored by loss fct during training
ignore_frames_loss = 6

In [4]:
classes = ['sparks', 'puffs', 'waves']

movie_ids = ["05","10","15","20","25","32","34","40","45"]
#movie_ids = ["05","34"]

In [5]:
dataset_dir = os.path.join("..", "data", "sparks_dataset")

## Load movies, annotations & predictions

In [6]:
# Config model to load
training_name = 'TEMP_new_annotated_peaks_physio'
epoch = 100000

preds_dir = os.path.join("trainings_validation", training_name)

In [7]:
### Load movies
xs = load_movies_ids(data_folder=dataset_dir,
                     ids=movie_ids,
                     names_available=True,
                     movie_names="video")

### Load annotations
# segmentation masks
ys = load_annotations_ids(data_folder=dataset_dir,
                          ids=movie_ids,
                          mask_names="class_label")

# event masks
ys_events = load_annotations_ids(data_folder=dataset_dir,
                                 ids=movie_ids,
                                 mask_names="event_label")

### Load predictions
sparks_filenames = {movie_id: os.path.join(preds_dir, f"{training_name}_{epoch}_{movie_id}_sparks.tif") for movie_id in movie_ids}
puffs_filenames = {movie_id: os.path.join(preds_dir, f"{training_name}_{epoch}_{movie_id}_puffs.tif") for movie_id in movie_ids}
waves_filenames = {movie_id: os.path.join(preds_dir, f"{training_name}_{epoch}_{movie_id}_waves.tif") for movie_id in movie_ids}

sparks = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in sparks_filenames.items()}
puffs = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in puffs_filenames.items()}
waves = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in waves_filenames.items()}

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


### Compute class weights for numbers of events

To use in order to get frames with the most annotated events in a weighted way

In [8]:
# define function to compute class weights
def compute_class_weights(ys, ys_events):
    '''
    Modified version of 'compute_class_weights' in  training_inference_tools.py
    (using numpy arrays instead of a SparkDataset instance, number of events
    instead of number of pixels and not considering background class)

    ys:         list of numpy arrays representing annotation masks with values between 0
                and 4
    ys_events:  list of numpy arrays representing event masks with int values

    returns:    dict of weights for each class
    '''

    # For 3 classes: sparks, puffs, waves
    w_dict = {'sparks': 0,
              'puffs': 0,
              'waves': 0}

    for y, y_events in zip(ys, ys_events):
        for event_type in w_dict.keys():
            # get binary mask with segmentation of type 'event_type'
            class_mask = y == class_to_nb(event_type)
            # get int mask with event instances of type 'event_type'
            event_mask = y_events * class_mask
            # count number of events of type 'event_type'
            count = np.unique(event_mask).size - 1

            w_dict[event_type] += count

    total = sum(w_dict.values())

    w_dict = {k: total/(3*v) for k, v in w_dict.items()}

    return w_dict

In [9]:
# compute class weights
w_dict = compute_class_weights(ys=list(ys.values()), ys_events=list(ys_events.values()))

In [10]:
w_dict

{'sparks': 0.43647798742138366,
 'puffs': 1.563063063063063,
 'waves': 14.458333333333334}

### Process predictions to obtain segmentation masks with small objects removed

In [11]:
# parameters necessary to process predictions

# physiological params (for spark peaks results)
pixel_size = 0.2 # 1 pixel = 0.2 um x 0.2 um
min_dist_xy = round(1.8 / pixel_size) # min distance in space
time_frame = 6.8 # 1 frame = 6.8 ms
min_dist_t = round(20 / time_frame) # min distance in time

# spark instances detection parameters
min_dist_xy = min_dist_xy
min_dist_t = min_dist_t
radius = math.ceil(min_dist_xy / 2)
y, x = np.ogrid[-radius : radius + 1, -radius : radius + 1]
disk = x**2 + y**2 <= radius**2
conn_mask = np.stack([disk] * (min_dist_t), axis=0)

# connectivity for event instances detection
connectivity = 26
sigma = 3

# TODO: use better parameters !!!
pixel_size = 0.2
spark_min_width = 3
spark_min_t = 3
puff_min_t = 5
wave_min_width = round(15 / pixel_size)

# maximal gap between two predicted puffs or waves that belong together
max_gap = 2  # i.e., 2 empty frames

In [12]:
preds_segmentation = {}

for movie_id in movie_ids:
    _, preds_segmentation[movie_id], _ = get_processed_result(
            sparks=sparks[movie_id],
            puffs=puffs[movie_id],
            waves=waves[movie_id],
            xs=xs[movie_id],
            conn_mask=conn_mask,
            connectivity=connectivity,
            max_gap=max_gap,
            sigma=sigma,
            wave_min_width=wave_min_width,
            puff_min_t=puff_min_t,
            spark_min_t=spark_min_t,
            spark_min_width=spark_min_width,
        )

## Compute statistics

In [13]:
# configure dataframe
cols = ['movie ID', 'frame ID', 
        'sparks IoU', 'puffs IoU', 'waves IoU', 'average IoU',
        'sparks events', 'puffs events', 'waves events',
        'sparks weighted events', 'puffs weighted events', 'waves weighted events', 
        'sum weighted events'
        ]

frames_stats_df = pd.DataFrame(columns=cols)

# iterate over movies and frame to compute statistics
for movie_id in movie_ids:
    # remove frames ignored by loss function
    sample_ys = empty_marginal_frames(video=ys[movie_id],
                                      n_frames=ignore_frames_loss)

    sample_ys_events = empty_marginal_frames(video=ys_events[movie_id],
                                             n_frames=ignore_frames_loss)
    
    # sample_preds is a dict with keys 'sparks','puffs' and 'waves'
    sample_preds = {} # where predicted segmentation will be stored
    for event_type in classes:
        sample_preds[event_type] = empty_marginal_frames(
                                    video=preds_segmentation[movie_id][event_type],
                                    n_frames=ignore_frames_loss
                                    )

    # mask ignored during training
    ignore_mask = sample_ys==class_to_nb('ignore')

    for frame_id in range(len(sample_ys)):
        # create frame dict that will be appended to dataframe
        frame_dict = {'movie ID': movie_id,
                      'frame ID': frame_id,
                      'average IoU' : 0,
                      'sum weighted events' : 0}

        for event_type in classes:
            # get binary masks corrisponding to event type
            ys_frame_class = sample_ys[frame_id]==class_to_nb(event_type)
            preds_frame_class = sample_preds[event_type][frame_id]

            # compute Intersection over Union score
            iou = compute_iou(ys_roi=ys_frame_class,
                              preds_roi=preds_frame_class,
                              ignore_mask=ignore_mask[frame_id])
            frame_dict[event_type+' IoU'] = iou
            frame_dict['average IoU'] += iou

            # get event mask corrisponding to event type
            ys_events_frame_class = np.where(ys_frame_class, sample_ys_events[frame_id], 0)
            # count number of events in frame
            frame_dict[event_type+' events'] = np.unique(ys_events_frame_class).size - 1
            # count number of weighted events in frame
            frame_dict[event_type+' weighted events'] = w_dict[event_type]*frame_dict[event_type+' events']
            frame_dict['sum weighted events'] += w_dict[event_type]*frame_dict[event_type+' events']
            
        frame_dict['average IoU'] /= len(classes)
        
        # create df from frame dict
        frame_df = pd.DataFrame([frame_dict])
        frames_stats_df = pd.concat([frames_stats_df, frame_df])

In [14]:
frames_stats_df

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,05,0,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,05,1,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,05,2,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,05,3,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,05,4,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,45,995,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,45,996,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,45,997,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,45,998,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0


### Discard empty frames (if both ys and preds are empty)

Idea: if nb pixels in annotations == 0 and IoU avg == 1 ==> nb pixels in preds = 0

In [15]:
frames_stats_df = frames_stats_df.loc[(frames_stats_df['average IoU']!=1) 
                                        | (frames_stats_df['sum weighted events']!=0)]

In [16]:
frames_stats_df

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,05,6,0.163488,0.0,0.0,0.054496,1,0,0,0.436478,0.0,0.0,0.436478
0,05,7,0.215539,0.0,0.0,0.071846,2,0,0,0.872956,0.0,0.0,0.872956
0,05,8,0.235012,0.0,0.0,0.078337,2,0,0,0.872956,0.0,0.0,0.872956
0,05,9,0.246341,0.0,1.0,0.415447,2,0,0,0.872956,0.0,0.0,0.872956
0,05,10,0.270270,0.0,1.0,0.423423,2,0,0,0.872956,0.0,0.0,0.872956
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,45,989,1.000000,1.0,0.0,0.666667,0,0,0,0.000000,0.0,0.0,0.000000
0,45,990,0.000000,1.0,0.0,0.333333,0,0,0,0.000000,0.0,0.0,0.000000
0,45,991,0.000000,1.0,0.0,0.333333,0,0,0,0.000000,0.0,0.0,0.000000
0,45,992,0.000000,1.0,0.0,0.333333,0,0,0,0.000000,0.0,0.0,0.000000


In [39]:
frames_stats_df.to_csv("C:/Users/dotti/OneDrive - Universitaet Bern/inter-rater_variability/frames_stats_df.csv", index=False)

## Select frames according to some criteria and visualize them in Napari

### Get frames where sparks IoU is worst, and annotated sparks aren't empty

In [21]:
# Get df of frames containing sparks
sparks_df = frames_stats_df.loc[frames_stats_df['sparks events']!=0]

In [22]:
# Order sparks df from worst sparks IoU to best
sparks_ordered_df = sparks_df.sort_values(['sparks IoU', 'sparks events'], ascending=[True, False])
sparks_ordered_df.head(10)

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,20,83,0.0,1.000000,1.0,0.666667,4,0,0,1.745912,0.000000,0.0,1.745912
0,20,84,0.0,1.000000,1.0,0.666667,4,0,0,1.745912,0.000000,0.0,1.745912
0,20,85,0.0,1.000000,1.0,0.666667,4,0,0,1.745912,0.000000,0.0,1.745912
0,15,54,0.0,0.359585,1.0,0.453195,3,1,0,1.309434,1.563063,0.0,2.872497
0,15,55,0.0,0.421053,1.0,0.473684,3,1,0,1.309434,1.563063,0.0,2.872497
0,15,56,0.0,0.405702,1.0,0.468567,3,1,0,1.309434,1.563063,0.0,2.872497
0,15,57,0.0,0.400433,1.0,0.466811,3,1,0,1.309434,1.563063,0.0,2.872497
0,15,80,0.0,0.548429,1.0,0.516143,3,1,0,1.309434,1.563063,0.0,2.872497
0,15,81,0.0,0.762500,1.0,0.587500,3,1,0,1.309434,1.563063,0.0,2.872497
0,15,114,0.0,1.000000,1.0,0.666667,3,0,0,1.309434,0.000000,0.0,1.309434


In [38]:
sparks_ordered_df.to_csv("C:/Users/dotti/OneDrive - Universitaet Bern/inter-rater_variability/sparks_ordered_df.csv", index=False)

### Get frames where puffs IoU is worst, and annotated puffs aren't empty

In [24]:
# Get df of frames containing puffs
puffs_df = frames_stats_df.loc[frames_stats_df['puffs events']!=0]

In [25]:
# Order puffs df from worst puffs IoU to best
puffs_ordered_df = puffs_df.sort_values(['puffs IoU', 'puffs events'], ascending=[True, False])
puffs_ordered_df.head(10)

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,20,445,0.000000,0.0,1.0,0.333333,0,4,0,0.000000,6.252252,0.0,6.252252
0,20,446,0.000000,0.0,1.0,0.333333,0,4,0,0.000000,6.252252,0.0,6.252252
0,20,447,0.000000,0.0,1.0,0.333333,0,4,0,0.000000,6.252252,0.0,6.252252
0,20,448,0.000000,0.0,1.0,0.333333,0,4,0,0.000000,6.252252,0.0,6.252252
0,20,449,0.000000,0.0,1.0,0.333333,0,4,0,0.000000,6.252252,0.0,6.252252
0,20,450,0.000000,0.0,1.0,0.333333,0,4,0,0.000000,6.252252,0.0,6.252252
0,20,451,0.000000,0.0,1.0,0.333333,0,4,0,0.000000,6.252252,0.0,6.252252
0,20,452,0.000000,0.0,1.0,0.333333,0,4,0,0.000000,6.252252,0.0,6.252252
0,20,453,0.000000,0.0,1.0,0.333333,0,4,0,0.000000,6.252252,0.0,6.252252
0,20,454,0.039216,0.0,1.0,0.346405,1,4,0,0.436478,6.252252,0.0,6.688730


In [37]:
puffs_ordered_df.to_csv("C:/Users/dotti/OneDrive - Universitaet Bern/inter-rater_variability/puffs_ordered_df.csv", index=False)

### Get frames where waves IoU is worst, and annotated waves aren't empty

In [27]:
# Get df of frames containing waves
waves_df = frames_stats_df.loc[frames_stats_df['waves events']!=0]

In [28]:
# Order waves df from worst waves IoU to best
waves_ordered_df = waves_df.sort_values(['waves IoU', 'waves events'], ascending=[True, False])
waves_ordered_df.head(10)

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,34,430,0.142857,0.0,0.0,0.047619,1,0,3,0.436478,0.0,43.375,43.811478
0,34,431,0.167421,0.0,0.0,0.055807,1,0,3,0.436478,0.0,43.375,43.811478
0,34,432,0.204124,0.0,0.0,0.068041,1,0,3,0.436478,0.0,43.375,43.811478
0,34,433,0.218692,0.0,0.0,0.072897,1,0,3,0.436478,0.0,43.375,43.811478
0,34,434,0.225979,0.0,0.0,0.075326,1,0,3,0.436478,0.0,43.375,43.811478
0,34,435,0.246073,0.0,0.0,0.082024,1,0,3,0.436478,0.0,43.375,43.811478
0,34,436,0.254833,0.0,0.0,0.084944,1,0,3,0.436478,0.0,43.375,43.811478
0,34,437,0.262206,0.0,0.0,0.087402,1,0,3,0.436478,0.0,43.375,43.811478
0,34,438,0.247104,0.0,0.0,0.082368,1,0,3,0.436478,0.0,43.375,43.811478
0,34,439,0.197158,0.0,0.0,0.065719,1,0,3,0.436478,0.0,43.375,43.811478


In [36]:
waves_ordered_df.to_csv("C:/Users/dotti/OneDrive - Universitaet Bern/inter-rater_variability/waves_ordered_df.csv", index=False)

### Get frames where average IoU is worst, and annotation aren't empty

In [31]:
# Get df of frames containing any events
events_df = frames_stats_df.loc[frames_stats_df['sum weighted events']!=0]

In [32]:
# Order events df from worst average IoU to best
events_ordered_df = events_df.sort_values(['average IoU', 'sum weighted events'], ascending=[True, False])
events_ordered_df.head(10)

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,34,580,0.0,0.0,0.0,0.0,1,0,2,0.436478,0.0,28.916667,29.353145
0,34,581,0.0,0.0,0.0,0.0,1,0,2,0.436478,0.0,28.916667,29.353145
0,34,582,0.0,0.0,0.0,0.0,1,0,2,0.436478,0.0,28.916667,29.353145
0,34,583,0.0,0.0,0.0,0.0,1,0,2,0.436478,0.0,28.916667,29.353145
0,34,590,0.0,0.0,0.0,0.0,1,0,2,0.436478,0.0,28.916667,29.353145
0,34,591,0.0,0.0,0.0,0.0,1,0,2,0.436478,0.0,28.916667,29.353145
0,34,592,0.0,0.0,0.0,0.0,1,0,2,0.436478,0.0,28.916667,29.353145
0,34,593,0.0,0.0,0.0,0.0,1,0,2,0.436478,0.0,28.916667,29.353145
0,34,594,0.0,0.0,0.0,0.0,1,0,2,0.436478,0.0,28.916667,29.353145
0,34,595,0.0,0.0,0.0,0.0,1,0,2,0.436478,0.0,28.916667,29.353145


In [35]:
events_ordered_df.to_csv("C:/Users/dotti/OneDrive - Universitaet Bern/inter-rater_variability/events_ordered_df.csv", index=False)

### For each movie, get frames with most weighted annotated events

In [34]:
for movie_id in movie_ids:
    most_annotated_df = frames_stats_df.loc[frames_stats_df['movie ID']==movie_id]
    most_annotated_df = most_annotated_df.sort_values('sum weighted events', ascending=False)
    display(most_annotated_df.head(5))
    most_annotated_df.to_csv((f"C:/Users/dotti/OneDrive - Universitaet Bern/inter-rater_variability/{str(movie_id)}_most_annotated_df.csv"), index=False)

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,05,256,0.189341,0.136134,1.0,0.441825,2,5,0,0.872956,7.815315,0.0,8.688271
0,05,254,0.222798,0.228782,1.0,0.483860,2,5,0,0.872956,7.815315,0.0,8.688271
0,05,255,0.199686,0.188172,1.0,0.462619,2,5,0,0.872956,7.815315,0.0,8.688271
0,05,99,0.240668,0.129105,1.0,0.456591,4,4,0,1.745912,6.252252,0.0,7.998164
0,05,100,0.268463,0.131303,1.0,0.466589,4,4,0,1.745912,6.252252,0.0,7.998164


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,10,413,0.0,0.220779,1.0,0.406926,2,5,0,0.872956,7.815315,0.0,8.688271
0,10,406,0.0,0.193106,1.0,0.397702,2,5,0,0.872956,7.815315,0.0,8.688271
0,10,412,0.0,0.208509,1.0,0.402836,2,5,0,0.872956,7.815315,0.0,8.688271
0,10,411,0.0,0.220542,1.0,0.406847,2,5,0,0.872956,7.815315,0.0,8.688271
0,10,410,0.0,0.217157,1.0,0.405719,2,5,0,0.872956,7.815315,0.0,8.688271


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,15,379,0.104364,0.360316,1.0,0.488227,2,4,0,0.872956,6.252252,0.0,7.125208
0,15,378,0.035565,0.350823,1.0,0.462129,2,4,0,0.872956,6.252252,0.0,7.125208
0,15,383,0.225303,0.390805,1.0,0.538703,2,4,0,0.872956,6.252252,0.0,7.125208
0,15,382,0.164311,0.371429,1.0,0.511913,2,4,0,0.872956,6.252252,0.0,7.125208
0,15,381,0.136201,0.355739,1.0,0.497313,2,4,0,0.872956,6.252252,0.0,7.125208


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,20,379,0.052101,0.203243,1.0,0.418448,3,4,0,1.309434,6.252252,0.0,7.561686
0,20,378,0.087344,0.226436,1.0,0.437927,3,4,0,1.309434,6.252252,0.0,7.561686
0,20,372,0.088319,0.325605,1.0,0.471308,3,4,0,1.309434,6.252252,0.0,7.561686
0,20,373,0.114078,0.309774,1.0,0.474617,3,4,0,1.309434,6.252252,0.0,7.561686
0,20,374,0.120729,0.291581,1.0,0.470770,3,4,0,1.309434,6.252252,0.0,7.561686


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,25,119,0.244403,0.000000,1.0,0.414801,3,1,0,1.309434,1.563063,0.0,2.872497
0,25,120,0.233209,0.000000,1.0,0.411070,3,1,0,1.309434,1.563063,0.0,2.872497
0,25,121,0.210728,0.000000,1.0,0.403576,3,1,0,1.309434,1.563063,0.0,2.872497
0,25,109,0.537356,0.329545,1.0,0.622301,2,1,0,0.872956,1.563063,0.0,2.436019
0,25,110,0.571816,0.240964,1.0,0.604260,2,1,0,0.872956,1.563063,0.0,2.436019


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,32,955,0.243243,0.0,1.0,0.414414,1,1,0,0.436478,1.563063,0.0,1.999541
0,32,880,0.364865,0.0,1.0,0.454955,1,1,0,0.436478,1.563063,0.0,1.999541
0,32,946,0.294118,0.0,1.0,0.431373,1,1,0,0.436478,1.563063,0.0,1.999541
0,32,947,0.535714,0.0,1.0,0.511905,1,1,0,0.436478,1.563063,0.0,1.999541
0,32,954,0.253521,0.0,1.0,0.417840,1,1,0,0.436478,1.563063,0.0,1.999541


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,34,894,0.395349,0.0,0.321839,0.239063,3,0,3,1.309434,0.0,43.375,44.684434
0,34,895,0.375486,0.0,0.314423,0.229970,2,0,3,0.872956,0.0,43.375,44.247956
0,34,893,0.386233,0.0,0.335000,0.240411,2,0,3,0.872956,0.0,43.375,44.247956
0,34,892,0.406072,0.0,0.333025,0.246366,2,0,3,0.872956,0.0,43.375,44.247956
0,34,897,0.046243,0.0,0.295366,0.113870,1,0,3,0.436478,0.0,43.375,43.811478


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,40,740,1.0,0.0,0.895143,0.631714,0,1,1,0.0,1.563063,14.458333,16.021396
0,40,898,0.0,0.0,0.617905,0.205968,0,1,1,0.0,1.563063,14.458333,16.021396
0,40,896,0.0,0.0,0.632625,0.210875,0,1,1,0.0,1.563063,14.458333,16.021396
0,40,895,1.0,0.0,0.636439,0.545480,0,1,1,0.0,1.563063,14.458333,16.021396
0,40,894,1.0,0.0,0.648589,0.549530,0,1,1,0.0,1.563063,14.458333,16.021396


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks events,puffs events,waves events,sparks weighted events,puffs weighted events,waves weighted events,sum weighted events
0,45,614,1.0,1.0,0.587715,0.862572,0,0,1,0.0,0.0,14.458333,14.458333
0,45,569,0.0,1.0,0.420065,0.473355,0,0,1,0.0,0.0,14.458333,14.458333
0,45,567,0.0,1.0,0.390134,0.463378,0,0,1,0.0,0.0,14.458333,14.458333
0,45,566,0.0,1.0,0.371475,0.457158,0,0,1,0.0,0.0,14.458333,14.458333
0,45,565,0.0,1.0,0.354870,0.451623,0,0,1,0.0,0.0,14.458333,14.458333


### Visualize sample movie with Napari

In [49]:
from napari.utils.theme import available_themes, get_theme
print('Originally themes', available_themes())

Originally themes ('dark', 'light', 'system')


In [64]:
movie_id = '45'
# get predicted segmentation with values between 0 and 3
preds = np.zeros_like(ys[movie_id])
for event_type in classes:
    preds += class_to_nb(event_type) * preds_segmentation[movie_id][event_type]

# get contours of annotated mask, for visualization
ys_contours = get_annotations_contour(annotations=ys[movie_id], contour_val=2)

In [65]:

viewer = napari.Viewer()
viewer.theme = 'dark'

viewer.add_image(xs[movie_id], 
                 name='original movie', 
                 colormap=('colors',cmap)
                )

viewer.add_labels(preds,
                  name='predicted segmentation',
                  opacity=0.5,
                  color=labels_cmap
                 )

viewer.add_labels(ys_contours,
                  name='annotated segmentation',
                  opacity=0.8,
                  color=labels_cmap
                 )

<Labels layer 'annotated segmentation' at 0x110ae09d3a0>